In [1]:
from google.colab import files
uploaded = files.upload()  # Upload your ZIP file

Saving Smart_City_Dataset.zip to Smart_City_Dataset.zip


In [3]:
from google.colab import files
geojson = files.upload()  # upload geojson

Saving city_map.geojson to city_map.geojson


In [4]:
import zipfile

# Automatically get the filename
zip_file = next(iter(uploaded))  # e.g., 'myfiles.zip'

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("unzipped")  # Extracts all files into /unzipped

In [5]:
import glob

# List all CSV files recursively
all_csv_files = glob.glob("unzipped/**/*.csv", recursive=True)
all_csv_files  # See all file paths

['unzipped/Essential_Data/weather_historical.csv',
 'unzipped/Essential_Data/sensor_readings.csv',
 'unzipped/Essential_Data/social_media_stream.csv',
 'unzipped/Essential_Data/disaster_events.csv',
 'unzipped/Distractor_Data/economic_activity.csv',
 'unzipped/Distractor_Data/transportation.csv',
 'unzipped/Distractor_Data/events_calendar.csv',
 'unzipped/Distractor_Data/energy_consumption.csv',
 'unzipped/Distractor_Data/local_business_reviews.csv']

# Visualize risk zones and live incident maps
# Recommend action plans

In [6]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
from sklearn.cluster import DBSCAN
import numpy as np

# --- Load GeoJSON Infrastructure ---
geojson_file = next(iter(geojson))
gdf = gpd.read_file(geojson_file).to_crs(epsg=4326)
gdf['centroid'] = gdf.geometry.centroid
centroids = gdf.set_geometry('centroid')

# --- Load Disaster Events ---
disaster_df = pd.read_csv('unzipped/Essential_Data/disaster_events.csv')
disaster_df['date'] = pd.to_datetime(disaster_df['date'])
recent_disasters = disaster_df.sort_values(by='date', ascending=False).head(50)

# --- Apply DBSCAN Clustering ---
coords = recent_disasters[['latitude', 'longitude']].to_numpy()
db = DBSCAN(eps=0.02, min_samples=2).fit(coords)  # ~2km radius
recent_disasters['cluster'] = db.labels_

# --- Setup Map ---
map_center = [recent_disasters['latitude'].mean(), recent_disasters['longitude'].mean()]
city_map = folium.Map(location=map_center, zoom_start=11)

# --- Disaster Colors ---
disaster_colors = {
    'fire': 'red',
    'earthquake': 'orange',
    'flood': 'blue',
    'industrial accident': 'purple',
    'hurricane': 'cadetblue'
}

# --- Smart Recommender with Far Flag ---
def generate_recommendation(dtype, zone, user_point):
    dtype = dtype.lower()
    rules = {
        "fire": {
            "types": ["fire_station", "hospital"],
            "action": "🚨 Fire in {zone}. Evacuate and dispatch emergency teams.",
            "label": "Fire Station or Hospital"
        },
        "flood": {
            "types": ["hospital", "shelter"],
            "action": "💧 Flood in {zone}. Reroute ambulances and open shelters.",
            "label": "Hospital or Shelter"
        },
        "earthquake": {
            "types": ["hospital"],
            "action": "🌍 Earthquake in {zone}. Assess damage and prepare hospitals.",
            "label": "Hospital"
        },
        "hurricane": {
            "types": ["shelter"],
            "action": "🌀 Hurricane in {zone}. Open shelters and secure property.",
            "label": "Shelter"
        },
        "industrial accident": {
            "types": ["hospital"],
            "action": "☣️ Industrial accident in {zone}. Evacuate and route to hospitals.",
            "label": "Hospital"
        }
    }

    if dtype not in rules:
        return f"🛑 Unknown disaster in {zone}. Monitor the situation closely.", "gray"

    rule = rules[dtype]
    filtered = gdf[gdf['type'].str.lower().isin(rule['types'])]

    if filtered.empty:
        return f"{rule['action'].format(zone=zone)} No matching infrastructure found.", "gray"

    filtered['distance'] = filtered.centroid.distance(user_point)
    nearest = filtered.sort_values(by='distance').iloc[0]

    name = nearest.get('name', 'Unknown')
    itype = nearest.get('type', 'Unknown')
    dist_km = round(nearest['distance'] * 111, 2)

    far_flag = "⚠️ (FAR)" if dist_km > 4 else ""
    note = "\n⚠️ Consider deploying mobile response or alerting backup units." if dist_km > 4 else ""

    icon_color = "orange" if dist_km > 4 else "green"

    msg = f"""{rule['action'].format(zone=zone)}
Nearest {rule['label']}: {name} ({itype}), approx. {dist_km} km away. {far_flag}{note}"""
    return msg, icon_color


# --- Plot Each Disaster + Response ---
for _, row in recent_disasters.iterrows():
    location = [row['latitude'], row['longitude']]
    zone = row['location']
    dtype = row['disaster_type'].lower()
    severity = row['severity']
    cluster = row['cluster']
    user_point = Point(row['longitude'], row['latitude'])

    # Cluster label for popup
    cluster_label = f"Cluster {cluster}" if cluster >= 0 else "Isolated"

    # Disaster circle marker
    folium.CircleMarker(
        location=location,
        radius=6 + severity,
        popup=f"{dtype.title()} in {zone} (Severity: {severity})\n{cluster_label}",
        color=disaster_colors.get(dtype, 'gray'),
        fill=True,
        fill_opacity=0.7
    ).add_to(city_map)

    # Response + distance-based marker
    response_text, icon_color = generate_recommendation(dtype, zone, user_point)
    folium.Marker(
        location=location,
        popup=response_text,
        icon=folium.Icon(color=icon_color, icon='info-sign')
    ).add_to(city_map)

# --- Show the map ---
city_map


<ipython-input-6-d2efc987588e>:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid
<ipython-input-6-d2efc987588e>:77: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  filtered['distance'] = filtered.centroid.distance(user_point)
<ipython-input-6-d2efc987588e>:77: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  filtered['distance'] = filtered.centroid.distance(user_point)
/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

### 🗺️ **Disaster Map Legend & Response Guide**

**🟠 Clusters:**  
Grouped incidents identified using **DBSCAN** clustering (within ~2km).  
Each cluster indicates a potential **hot zone** with multiple nearby disasters.

---

**🔴 Disaster Circle Markers (Color by Type):**

| Color         | Disaster Type         |
|---------------|------------------------|
| 🔴 Red        | Fire                  |
| 🔵 Blue       | Flood                 |
| 🟣 Purple     | Industrial Accident   |
| 🟠 Orange     | Earthquake            |
| 🟦 Cadet Blue | Hurricane             |

---

**🟢/🟠 Info Icon Markers (Recommendations):**

| Icon Color     | Meaning                                                |
|----------------|--------------------------------------------------------|
| 🟢 Green       | Infrastructure found nearby (**within 4 km**)         |
| 🟠 Orange      | ⚠️ Far infrastructure (**beyond 4 km**) — possible delay |
| ⚠️ Flag        | Indicates delayed or distant response (added in text)  |

---

**📌 Popup Contents Include:**
- Disaster type, location (zone), and severity level
- Cluster number or "Isolated"
- LLM-style recommendation:
  - Suggested action (e.g., evacuate, reroute)
  - Nearest relevant facility (e.g., Fire Station, Hospital, Shelter)
  - Distance from incident

---

**🚑 Infrastructure Types Used:**

- `fire_station` – Emergency fire response
- `hospital` – Medical support & triage
- `shelter` – Evacuation centers during flood or hurricane

---

> 🧠 *This map is powered by spatial clustering and disaster-aware logic, combining real-time incident data with infrastructure intelligence.*
